# Imputation Methods
by Wilmer Garzón, last updated: 30-June-2025

This notebook demonstrates various imputation methods for handling missing data. We will use a sample dataset with missing values and apply different techniques to fill in the gaps. Finally, we will compare the methods using Mean Squared Error (MSE).

In [1]:
import pandas as pd
import numpy as np
#Imports KNNImputer to fill missing values using nearest neighbors.
from sklearn.impute import KNNImputer
#Imports LinearRegression to build linear models
from sklearn.linear_model import LinearRegression
#Imports mean_squared_error to evaluate prediction accuracy.
from sklearn.metrics import mean_squared_error
#Imports train_test_split to divide data into training and test sets.
from sklearn.model_selection import train_test_split

## Dataset Description:
- **Number of samples**: 500
- **Number of features**: 6 numerical features
- **Variables**: Feature_1, Feature_2, ..., Feature_6
- **Target variable**: Target
- **Missing values**: None, the dataset is fully complete

In [26]:
# Load the dataset
data_org = pd.read_csv('complete_dataset.csv')
data_org.head()

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Target
0,57.450712,47.926035,59.715328,72.845448,46.487699,46.487946,377.845002
1,73.688192,61.511521,42.957884,58.138401,43.048735,43.014054,401.933827
2,53.629434,21.300796,24.126233,41.565687,34.807533,54.713710,266.092906
3,36.379639,28.815444,71.984732,46.613355,51.012923,28.628777,272.331966
4,41.834259,51.663839,32.735096,55.635470,40.990420,45.624594,320.151692


In [27]:
# Check for missing values
data_org.isnull().sum()

,0
Feature_1,0
Feature_2,0
Feature_3,0
Feature_4,0
Feature_5,0
Feature_6,0
Target,0


## Exercises:
- Please complete the following tasks step by step.
- Each numbered item corresponds to a specific imputation technique or evaluation method that you must implement and analyze.

**STEP 0: INTRODUCE 10% OF MISSING VALUES**

In [32]:
data = data_org.copy()
for _ in range(int(0.10*data_org.size)):
  row = np.random.randint(0,data.shape[0])
  col = np.random.randint(0,data.shape[1])
  data.iat[row, col] = np.nan

data.isnull().sum()


,0
Feature_1,37
Feature_2,53
Feature_3,46
Feature_4,47
Feature_5,46
Feature_6,50
Target,56


### Imputation with Mean
*Impute missing values by replacing them with the mean of each feature*

In [33]:
mean_data = data.copy()
for col in mean_data.columns:
  mean_data[col] = mean_data[col].fillna(mean_data[col].mean())


mean_data.isnull().sum()

,0
Feature_1,0
Feature_2,0
Feature_3,0
Feature_4,0
Feature_5,0
Feature_6,0
Target,0


### Imputation with Median
*Impute missing values using the median of each feature to reduce the impact of outliers*

In [34]:
median_data = data.copy()
for col in median_data.columns:
  median_data[col] = median_data[col].fillna(median_data[col].median())

median_data.isnull().sum()

,0
Feature_1,0
Feature_2,0
Feature_3,0
Feature_4,0
Feature_5,0
Feature_6,0
Target,0


### Imputation with KNN
*Use the K-Nearest Neighbors (KNN) algorithm to impute missing values based on similar observations*

In [67]:
knn_data = data.copy()
imputer = KNNImputer(n_neighbors=15)
knn_data = pd.DataFrame(imputer.fit_transform(knn_data), columns=knn_data.columns)

knn_data.isnull().sum()

,0
Feature_1,0
Feature_2,0
Feature_3,0
Feature_4,0
Feature_5,0
Feature_6,0
Target,0


### Imputation with Linear Regression
*Train a linear regression model on the available data and use it to predict and fill in missing values*

In [54]:
from sklearn.impute import SimpleImputer

LR_data = data.copy()
features = LR_data.columns.drop("Target")
imputer = SimpleImputer(strategy="mean")
LR_data[features] = imputer.fit_transform(LR_data[features])
missing = LR_data["Target"].isnull()
not_missing = ~missing

X_train = LR_data.loc[not_missing, features]
y_train = data.loc[not_missing, "Target"]
X_pred = LR_data.loc[missing, features]

model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_pred)

LR_data.loc[missing, "Target"] = predictions

LR_data.isnull().sum()


,0
Feature_1,0
Feature_2,0
Feature_3,0
Feature_4,0
Feature_5,0
Feature_6,0
Target,0


### Mean Squared Error (MSE)
- The MSE is a common metric used to evaluate the accuracy of a model or method by measuring the average of the squares of the errors.
- The average squared difference between the actual (true) values and the predicted or imputed values.

![MSE](https://miro.medium.com/v2/resize:fit:720/format:webp/0*ox49JmZ2YkKrqG9N.jpg)

### Compare Imputation Methods using MSE
*Evaluate and compare imputation methods by calculating the Mean Squared Error (MSE) between imputed and true values*

In [68]:
missing_target_idx = data['Target'].isnull()


mse_results = {
    'Mean': mean_squared_error(data_org, mean_data),
    'Median': mean_squared_error(data_org, median_data),
    'KNN': mean_squared_error(data_org, knn_data),
    'Linear Regression': mean_squared_error(data_org, LR_data)
}

mse_df = pd.DataFrame.from_dict(mse_results, orient='index', columns=['MSE'])
mse_df = mse_df.sort_values('MSE')

mse_df.reset_index().rename(columns={'index': 'Method'})

,Method,MSE
0,Linear Regression,19.876032
1,KNN,23.891737
2,Mean,45.555532
3,Median,46.003182
